<a href="https://colab.research.google.com/github/shreyaghotankar/CMPE-255-Data-Mining/blob/master/CMPE255_FrequentPatternMining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE 255 - Assignment - Frequent Pattern Mining Apriori and FPGrowth

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install apyori

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from apyori import apriori

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

## Data Analysis and Cleaning

Dataset - [e-commerce dataset](https://www.kaggle.com/carrie1/ecommerce-data)

In [8]:
df = pd.read_csv('/content/drive/MyDrive/CMPE255 - Data Mining/Assignments/Datasets/data.csv', encoding='latin-1')

In [9]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [10]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [11]:
df.shape

(541909, 8)

In [12]:
print(df.columns)

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


In [13]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [14]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [15]:
df.dropna(axis=0, inplace=True)
df.shape

(406829, 8)

In [16]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Greece', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Saudi Arabia', 'Czech Republic', 'Canada',
       'Unspecified', 'Brazil', 'USA', 'European Community', 'Bahrain',
       'Malta', 'RSA'], dtype=object)

In [17]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df[df['InvoiceNo'].str.contains('C')] #canceled orders

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


In [18]:
df = df[~df['InvoiceNo'].str.contains('C')]
df.shape

(397924, 8)

In [19]:
df[df['Country'].str.contains('USA')].count() # checking for country type

InvoiceNo      179
StockCode      179
Description    179
Quantity       179
InvoiceDate    179
UnitPrice      179
CustomerID     179
Country        179
dtype: int64

In [20]:
df_USA = df[df['Country'].str.contains('USA')].copy()
df_USA.shape

(179, 8)

In [21]:
df_USA['Description'] = df_USA['Description'].str.strip()

In [22]:
data = pd.get_dummies(df_USA['Description'])
data.head()

,12 PENCILS SMALL TUBE RED RETROSPOT,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,6 GIFT TAGS 50'S CHRISTMAS,6 GIFT TAGS VINTAGE CHRISTMAS,60 CAKE CASES DOLLY GIRL DESIGN,60 TEATIME FAIRY CAKE CASES,BAKING SET 9 PIECE RETROSPOT,BIRTHDAY BANQUET GIFT WRAP,BLUE HAPPY BIRTHDAY BUNTING,BLUE POLKADOT BOWL,BLUE POLKADOT CUP,BLUE POLKADOT WRAP,BLUE STRIPE CERAMIC DRAWER KNOB,BREAD BIN DINER STYLE PINK,CARD BILLBOARD FONT,CARD CHRISTMAS VILLAGE,CARD DOLLY GIRL,CERAMIC CAKE DESIGN SPOTTED MUG,CHILDREN'S SPACEBOY MUG,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY DOLLY GIRL,CHILDRENS CUTLERY POLKADOT BLUE,CHILDRENS CUTLERY POLKADOT GREEN,CHILDRENS CUTLERY POLKADOT PINK,CHILDRENS CUTLERY RETROSPOT RED,CHILDRENS CUTLERY SPACEBOY,CHILDRENS DOLLY GIRL MUG,COFFEE MUG CAT + BIRD DESIGN,COTTON APRON PANTRY DESIGN,CUPCAKE LACE PAPER SET 6,DOLLY GIRL CHILDRENS BOWL,DOLLY GIRL CHILDRENS CUP,DOORMAT FAIRY CAKE,DOORMAT SPOTTY HOME SWEET HOME,DOUGHNUT LIP GLOSS,ELEPHANT BIRTHDAY CARD,EMBROIDERED RIBBON REEL DAISY,EMBROIDERED RIBBON REEL RACHEL,EMBROIDERED RIBBON REEL ROSIE,EMBROIDERED RIBBON REEL SALLY,EMBROIDERED RIBBON REEL SOPHIE,EMBROIDERED RIBBON REEL SUSIE,FAIRY CAKE BIRTHDAY CANDLE SET,FELTCRAFT 6 FLOWER FRIENDS,FELTCRAFT BUTTERFLY HEARTS,FRIDGE MAGNETS US DINER ASSORTED,FRYING PAN RED RETROSPOT,GLASS APOTHECARY BOTTLE ELIXIR,GLASS JAR ENGLISH CONFECTIONERY,GREEN POLKADOT BOWL,GREEN POLKADOT PLATE,ICE CREAM PEN LIP GLOSS,JAM JAR WITH GREEN LID,JAM JAR WITH PINK LID,JAM MAKING SET PRINTED,JUMBO BAG 50'S CHRISTMAS,JUMBO BAG RED RETROSPOT,JUNGLE POPSICLES ICE LOLLY MOULDS,LARGE CERAMIC TOP STORAGE JAR,LONDON BUS COFFEE MUG,LUNCH BAG VINTAGE DOILY,MAGNETS PACK OF 4 CHILDHOOD MEMORY,MEDIUM CERAMIC TOP STORAGE JAR,MILK PAN BLUE POLKADOT,MINI PAINT SET VINTAGE,PACK OF 20 NAPKINS PANTRY DESIGN,PACK OF 60 MUSHROOM CAKE CASES,PACK OF 60 SPACEBOY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,PANTRY WASHING UP BRUSH,PAPER BUNTING COLOURED LACE,PAPER BUNTING RETROSPOT,PAPER CHAIN KIT RETROSPOT,PARTY CONES CANDY ASSORTED,PARTY CONES CANDY TREE DECORATION,PARTY INVITES DINOSAURS,PARTY INVITES WOODLAND,PINK POLKADOT CUP,PINK BABY BUNTING,PINK HAPPY BIRTHDAY BUNTING,PINK PAISLEY ROSE GIFT WRAP,PINK POLKADOT BOWL,PINK POLKADOT PLATE,PLASTERS IN TIN SPACEBOY,RED HARMONICA IN BOX,RED RETROSPOT BOWL,RED RETROSPOT CUP,RED RETROSPOT MINI CASES,RED RETROSPOT PLATE,RED RETROSPOT WRAP,RED SPOT CERAMIC DRAWER KNOB,RED STRIPE CERAMIC DRAWER KNOB,REGENCY CAKESTAND 3 TIER,REGENCY MILK JUG PINK,REGENCY SUGAR BOWL GREEN,REGENCY TEAPOT ROSES,RETRO PLASTIC ELEPHANT TRAY,RETROSPOT TEA SET CERAMIC 11 PC,RIBBON REEL CHRISTMAS PRESENT,RIBBON REEL CHRISTMAS SOCK BAUBLE,RIBBON REEL MAKING SNOWMEN,RIBBON REEL SNOWY VILLAGE,ROLL WRAP 50'S CHRISTMAS,ROLL WRAP VINTAGE CHRISTMAS,ROUND SNACK BOXES SET OF4 WOODLAND,SAVE THE PLANET COTTON TOTE BAG,SET 12 COLOUR PENCILS SPACEBOY,SET 12 COLOURING PENCILS DOILY,SET 2 PANTRY DESIGN TEA TOWELS,SET 2 TEA TOWELS I LOVE LONDON,SET 36 COLOUR PENCILS DOLLY GIRL,SET 4 PICNIC CUTLERY BLUEBERRY,SET 4 PICNIC CUTLERY CHERRY,SET 4 PICNIC CUTLERY FONDANT,SET 6 SCHOOL MILK BOTTLES IN CRATE,SET OF 12 FAIRY CAKE BAKING CASES,SET OF 12 MINI LOAF BAKING CASES,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 3 REGENCY CAKE TINS,SET OF 36 PAISLEY FLOWER DOILIES,SET OF 36 TEATIME PAPER DOILIES,SET OF 4 ENGLISH ROSE PLACEMATS,SET OF 4 PANTRY JELLY MOULDS,SET OF 5 LUCKY CAT MAGNETS,SET OF 5 MINI GROCERY MAGNETS,SET OF 5 PANCAKE DAY MAGNETS,SET OF 6 RIBBONS VINTAGE CHRISTMAS,SET OF 6 SNACK LOAF BAKING CASES,SET OF 6 SOLDIER SKITTLES,SET OF 6 SPICE TINS PANTRY DESIGN,SET OF 6 TEA TIME BAKING CASES,SET OF 60 I LOVE LONDON CAKE CASES,SET OF 72 RETROSPOT PAPER DOILIES,SET/10 BLUE POLKADOT PARTY CANDLES,SET/10 IVORY POLKADOT PARTY CANDLES,SET/10 PINK POLKADOT PARTY CANDLES,SET/10 RED POLKADOT PARTY CANDLES,SILK PURSE BABUSHKA PINK,SMALL CERAMIC TOP STORAGE JAR,SPACEBOY BIRTHDAY CARD,SPACEBOY CHILDRENS BOWL,SPACEBOY CHILDRENS CUP,SPACEBOY 

In [23]:
df_data = df_USA.drop(['Description'], axis=1)
df_data = df_data.join(data)
df_data.head()

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,12 PENCILS SMALL TUBE RED RETROSPOT,3 RAFFIA RIBBONS 50'S CHRISTMAS,3 TRADITIONAl BISCUIT CUTTERS SET,36 DOILIES DOLLY GIRL,6 GIFT TAGS 50'S CHRISTMAS,6 GIFT TAGS VINTAGE CHRISTMAS,60 CAKE CASES DOLLY GIRL DESIGN,60 TEATIME FAIRY CAKE CASES,BAKING SET 9 PIECE RETROSPOT,BIRTHDAY BANQUET GIFT WRAP,BLUE HAPPY BIRTHDAY BUNTING,BLUE POLKADOT BOWL,BLUE POLKADOT CUP,BLUE POLKADOT WRAP,BLUE STRIPE CERAMIC DRAWER KNOB,BREAD BIN DINER STYLE PINK,CARD BILLBOARD FONT,CARD CHRISTMAS VILLAGE,CARD DOLLY GIRL,CERAMIC CAKE DESIGN SPOTTED MUG,CHILDREN'S SPACEBOY MUG,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY DOLLY GIRL,CHILDRENS CUTLERY POLKADOT BLUE,CHILDRENS CUTLERY POLKADOT GREEN,CHILDRENS CUTLERY POLKADOT PINK,CHILDRENS CUTLERY RETROSPOT RED,CHILDRENS CUTLERY SPACEBOY,CHILDRENS DOLLY GIRL MUG,COFFEE MUG CAT + BIRD DESIGN,COTTON APRON PANTRY DESIGN,CUPCAKE LACE PAPER SET 6,DOLLY GIRL CHILDRENS BOWL,DOLLY GIRL CHILDRENS CUP,DOORMAT FAIRY CAKE,DOORMAT SPOTTY HOME SWEET HOME,DOUGHNUT LIP GLOSS,ELEPHANT BIRTHDAY CARD,EMBROIDERED RIBBON REEL DAISY,EMBROIDERED RIBBON REEL RACHEL,EMBROIDERED RIBBON REEL ROSIE,EMBROIDERED RIBBON REEL SALLY,EMBROIDERED RIBBON REEL SOPHIE,EMBROIDERED RIBBON REEL SUSIE,FAIRY CAKE BIRTHDAY CANDLE SET,FELTCRAFT 6 FLOWER FRIENDS,FELTCRAFT BUTTERFLY HEARTS,FRIDGE MAGNETS US DINER ASSORTED,FRYING PAN RED RETROSPOT,GLASS APOTHECARY BOTTLE ELIXIR,GLASS JAR ENGLISH CONFECTIONERY,GREEN POLKADOT BOWL,GREEN POLKADOT PLATE,ICE CREAM PEN LIP GLOSS,JAM JAR WITH GREEN LID,JAM JAR WITH PINK LID,JAM MAKING SET PRINTED,JUMBO BAG 50'S CHRISTMAS,JUMBO BAG RED RETROSPOT,JUNGLE POPSICLES ICE LOLLY MOULDS,LARGE CERAMIC TOP STORAGE JAR,LONDON BUS COFFEE MUG,LUNCH BAG VINTAGE DOILY,MAGNETS PACK OF 4 CHILDHOOD MEMORY,MEDIUM CERAMIC TOP STORAGE JAR,MILK PAN BLUE POLKADOT,MINI PAINT SET VINTAGE,PACK OF 20 NAPKINS PANTRY DESIGN,PACK OF 60 MUSHROOM CAKE CASES,PACK OF 60 SPACEBOY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,PANTRY WASHING UP BRUSH,PAPER BUNTING COLOURED LACE,PAPER BUNTING RETROSPOT,PAPER CHAIN KIT RETROSPOT,PARTY CONES CANDY ASSORTED,PARTY CONES CANDY TREE DECORATION,PARTY INVITES DINOSAURS,PARTY INVITES WOODLAND,PINK POLKADOT CUP,PINK BABY BUNTING,PINK HAPPY BIRTHDAY BUNTING,PINK PAISLEY ROSE GIFT WRAP,PINK POLKADOT BOWL,PINK POLKADOT PLATE,PLASTERS IN TIN SPACEBOY,RED HARMONICA IN BOX,RED RETROSPOT BOWL,RED RETROSPOT CUP,RED RETROSPOT MINI CASES,RED RETROSPOT PLATE,RED RETROSPOT WRAP,RED SPOT CERAMIC DRAWER KNOB,RED STRIPE CERAMIC DRAWER KNOB,REGENCY CAKESTAND 3 TIER,REGENCY MILK JUG PINK,REGENCY SUGAR BOWL GREEN,REGENCY TEAPOT ROSES,RETRO PLASTIC ELEPHANT TRAY,RETROSPOT TEA SET CERAMIC 11 PC,RIBBON REEL CHRISTMAS PRESENT,RIBBON REEL CHRISTMAS SOCK BAUBLE,RIBBON REEL MAKING SNOWMEN,RIBBON REEL SNOWY VILLAGE,ROLL WRAP 50'S CHRISTMAS,ROLL WRAP VINTAGE CHRISTMAS,ROUND SNACK BOXES SET OF4 WOODLAND,SAVE THE PLANET COTTON TOTE BAG,SET 12 COLOUR PENCILS SPACEBOY,SET 12 COLOURING PENCILS DOILY,SET 2 PANTRY DESIGN TEA TOWELS,SET 2 TEA TOWELS I LOVE LONDON,SET 36 COLOUR PENCILS DOLLY GIRL,SET 4 PICNIC CUTLERY BLUEBERRY,SET 4 PICNIC CUTLERY CHERRY,SET 4 PICNIC CUTLERY FONDANT,SET 6 SCHOOL MILK BOTTLES IN CRATE,SET OF 12 FAIRY CAKE BAKING CASES,SET OF 12 MINI LOAF BAKING CASES,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 3 REGENCY CAKE TINS,SET OF 36 PAISLEY FLOWER DOILIES,SET OF 36 TEATIME PAPER DOILIES,SET OF 4 ENGLISH ROSE PLACEMATS,SET OF 4 PANTRY JELLY MOULDS,SET OF 5 LUCKY CAT MAGNETS,SET OF 5 MINI GROCERY MAGNETS,SET OF 5 PANCAKE DAY MAGNETS,SET OF 6 RIBBONS VINTAGE CHRISTMAS,SET OF 6 SNACK LOAF BAKING CASES,SET OF 6 SOLDIER SKITTLES,SET OF 6 SPICE TINS PANTRY DESIGN,SET OF 6 TEA TIME BAKING CASES,SET OF 60 I LOVE LONDON CAKE CASES,SET OF 72 RETROSPOT PAPER DOILIES,SET/10 BLUE POLKADOT PARTY CANDLES,SET/10 IVORY POLKADOT PARTY CANDLES,SET/10 PINK POLKADOT PARTY CANDLES,SET/10 RED POLKADOT PARTY CANDLES,SILK PURSE BABUSHKA PINK,SMALL CERAMIC TOP STORAGE JAR,SPACEBOY 

In [24]:
prods = df_USA['Description'].unique()
prods

array(['SET OF 6 SPICE TINS PANTRY DESIGN', 'PANTRY WASHING UP BRUSH',
       'SET OF 36 TEATIME PAPER DOILIES',
       'SET OF 3 CAKE TINS PANTRY DESIGN', 'SET OF 4 PANTRY JELLY MOULDS',
       'TEA TIME DES TEA COSY', 'SET 2 PANTRY DESIGN TEA TOWELS',
       'SET 2 TEA TOWELS I LOVE LONDON', 'PARTY CONES CANDY ASSORTED',
       'PINK HAPPY BIRTHDAY BUNTING',
       '3 TRADITIONAl BISCUIT CUTTERS  SET',
       'PAPER BUNTING COLOURED LACE', 'COTTON APRON PANTRY DESIGN',
       'PARTY CONES CANDY TREE DECORATION', 'VINTAGE RED TEATIME MUG',
       'FELTCRAFT 6 FLOWER FRIENDS', 'FELTCRAFT BUTTERFLY HEARTS',
       'COFFEE MUG CAT + BIRD DESIGN', 'LONDON BUS COFFEE MUG',
       'REGENCY CAKESTAND 3 TIER', 'TEA TIME TEA SET IN GIFT BOX',
       'TEA TIME TEAPOT IN GIFT BOX', 'PLASTERS IN TIN SPACEBOY',
       'RED  HARMONICA IN BOX', 'SET OF 6 SOLDIER SKITTLES',
       'BAKING SET 9 PIECE RETROSPOT', 'DOORMAT SPOTTY HOME SWEET HOME',
       'RED STRIPE CERAMIC DRAWER KNOB', 'RED SPOT CERA

In [33]:
df_filtered = df_data.groupby(['CustomerID','InvoiceNo'])[prods[:]].sum()
df_filtered

SET OF 6 SPICE TINS PANTRY DESIGN  \
CustomerID InvoiceNo                                      
12558.0    580158                                     0   
12607.0    570467                                     0   
12646.0    572215                                     0   
           580553                                     0   
12733.0    550644                                     1   

                      PANTRY WASHING UP BRUSH  \
CustomerID InvoiceNo                            
12558.0    580158                           0   
12607.0    570467                           0   
12646.0    572215                           0   
           580553                           0   
12733.0    550644                           1   

                      SET OF 36 TEATIME PAPER DOILIES  \
CustomerID InvoiceNo                                    
12558.0    580158                                   0   
12607.0    570467                                   1   
12646.0    572215                                   0   
           580553                                   0   
12733.0    550644                                   1   

                      SET OF 3 CAKE TINS PANTRY DESIGN  \
CustomerID InvoiceNo                                     
12558.0    580158                                    0   
12607.0    570467                                    1   
12646.0    572215                                    0   
           580553                                    0   
12733.0    550644                                    1   

                      SET OF 4 PANTRY JELLY MOULDS  TEA TIME DES TEA COSY  \
CustomerID InvoiceNo                                                        
12558.0    580158                                0                      0   
12607.0    570467                                0                      0   
12646.0    572215                                0                      0   
           580553                                0                      0   
12733.0    550644                                1                      1   

                      SET 2 PANTRY DESIGN TEA TOWELS  \
CustomerID InvoiceNo                                   
12558.0    580158                                  0   
12607.0    570467                                  1   
12646.0    572215                                  0   
           580553                                  0   
12733.0    550644                                  1   

                      SET 2 TEA TOWELS I LOVE LONDON  \
CustomerID InvoiceNo                                   
12558.0    580158                                  0   
12607.0    570467                                  0   
12646.0    572215                                  0   
           580553                                  0   
12733.0    550644                                  1   

                      PARTY CONES CANDY ASSORTED  PINK HAPPY BIRTHDAY BUNTING  \
CustomerID InvoiceNo                                                            
12558.0    580158                              0                            0   
12607.0    570467                              0                            1   
12646.0    572215                              0                            0   
           580553                              0                            0   
12733.0    550644                              1                            1   

                      3 TRADITIONAl BISCUIT CUTTERS  SET  \
CustomerID InvoiceNo                                       
12558.0    580158                                      0   
12607.0    570467                                      0   
12646.0    572215                                      0   
           580553                                      0   
12733.0    550644                                      1   

                      PAPER BUNTING COLOURED LACE  COTTON APRON PANTRY DESIGN  \
CustomerID InvoiceNo                                         

In [34]:
df_filtered = df_filtered.reset_index()[prods]
df_filtered.head()

,SET OF 6 SPICE TINS PANTRY DESIGN,PANTRY WASHING UP BRUSH,SET OF 36 TEATIME PAPER DOILIES,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 4 PANTRY JELLY MOULDS,TEA TIME DES TEA COSY,SET 2 PANTRY DESIGN TEA TOWELS,SET 2 TEA TOWELS I LOVE LONDON,PARTY CONES CANDY ASSORTED,PINK HAPPY BIRTHDAY BUNTING,3 TRADITIONAl BISCUIT CUTTERS SET,PAPER BUNTING COLOURED LACE,COTTON APRON PANTRY DESIGN,PARTY CONES CANDY TREE DECORATION,VINTAGE RED TEATIME MUG,FELTCRAFT 6 FLOWER FRIENDS,FELTCRAFT BUTTERFLY HEARTS,COFFEE MUG CAT + BIRD DESIGN,LONDON BUS COFFEE MUG,REGENCY CAKESTAND 3 TIER,TEA TIME TEA SET IN GIFT BOX,TEA TIME TEAPOT IN GIFT BOX,PLASTERS IN TIN SPACEBOY,RED HARMONICA IN BOX,SET OF 6 SOLDIER SKITTLES,BAKING SET 9 PIECE RETROSPOT,DOORMAT SPOTTY HOME SWEET HOME,RED STRIPE CERAMIC DRAWER KNOB,RED SPOT CERAMIC DRAWER KNOB,BLUE STRIPE CERAMIC DRAWER KNOB,PAPER BUNTING RETROSPOT,PINK BABY BUNTING,BLUE HAPPY BIRTHDAY BUNTING,CHILDRENS DOLLY GIRL MUG,CHILDREN'S SPACEBOY MUG,ROUND SNACK BOXES SET OF4 WOODLAND,SWEETHEART CERAMIC TRINKET BOX,STRAWBERRY CERAMIC TRINKET BOX,36 DOILIES DOLLY GIRL,SET OF 36 PAISLEY FLOWER DOILIES,SET OF 4 ENGLISH ROSE PLACEMATS,CERAMIC CAKE DESIGN SPOTTED MUG,STRAWBERRY FAIRY CAKE TEAPOT,60 CAKE CASES DOLLY GIRL DESIGN,PACK OF 60 MUSHROOM CAKE CASES,PACK OF 60 SPACEBOY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,CUPCAKE LACE PAPER SET 6,60 TEATIME FAIRY CAKE CASES,FRYING PAN RED RETROSPOT,BREAD BIN DINER STYLE PINK,MILK PAN BLUE POLKADOT,GLASS JAR ENGLISH CONFECTIONERY,TEA TIME OVEN GLOVE,MAGNETS PACK OF 4 CHILDHOOD MEMORY,FRIDGE MAGNETS US DINER ASSORTED,SET OF 5 MINI GROCERY MAGNETS,SET OF 5 PANCAKE DAY MAGNETS,TEA PARTY BIRTHDAY CARD,SPACEBOY BIRTHDAY CARD,CARD BILLBOARD FONT,CARD DOLLY GIRL,WRAP ENGLISH ROSE,RED RETROSPOT WRAP,BLUE POLKADOT WRAP,TEA PARTY WRAPPING PAPER,BIRTHDAY BANQUET GIFT WRAP,PINK PAISLEY ROSE GIFT WRAP,WRAP DOLLY GIRL,WRAP COWBOYS,SET/10 PINK POLKADOT PARTY CANDLES,SET/10 IVORY POLKADOT PARTY CANDLES,TRADITIONAL WOODEN SKIPPING ROPE,MEDIUM CERAMIC TOP STORAGE JAR,LARGE CERAMIC TOP STORAGE JAR,SMALL CERAMIC TOP STORAGE JAR,DOORMAT FAIRY CAKE,SET 36 COLOUR PENCILS DOLLY GIRL,SET 12 COLOUR PENCILS SPACEBOY,EMBROIDERED RIBBON REEL SUSIE,EMBROIDERED RIBBON REEL RACHEL,EMBROIDERED RIBBON REEL SALLY,RETROSPOT TEA SET CERAMIC 11 PC,ICE CREAM PEN LIP GLOSS,DOUGHNUT LIP GLOSS,PARTY INVITES WOODLAND,PARTY INVITES DINOSAURS,MINI PAINT SET VINTAGE,RIBBON REEL SNOWY VILLAGE,SET OF 6 RIBBONS VINTAGE CHRISTMAS,RIBBON REEL CHRISTMAS PRESENT,3 RAFFIA RIBBONS 50'S CHRISTMAS,RIBBON REEL MAKING SNOWMEN,RIBBON REEL CHRISTMAS SOCK BAUBLE,WRAP CHRISTMAS VILLAGE,CARD CHRISTMAS VILLAGE,GLASS APOTHECARY BOTTLE ELIXIR,VINTAGE DONKEY TAIL GAME,DOLLY GIRL CHILDRENS BOWL,DOLLY GIRL CHILDRENS CUP,SPACEBOY CHILDRENS BOWL,SPACEBOY CHILDRENS CUP,SET OF 72 RETROSPOT PAPER DOILIES,6 GIFT TAGS VINTAGE CHRISTMAS,ROLL WRAP VINTAGE CHRISTMAS,ROLL WRAP 50'S CHRISTMAS,6 GIFT TAGS 50'S CHRISTMAS,VINTAGE CHRISTMAS STOCKING,SET OF 60 I LOVE LONDON CAKE CASES,SET OF 12 FAIRY CAKE BAKING CASES,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY SPACEBOY,CHILDRENS CUTLERY DOLLY GIRL,RED RETROSPOT MINI CASES,SILK PURSE BABUSHKA PINK,SAVE THE PLANET COTTON TOTE BAG,FAIRY CAKE BIRTHDAY CANDLE SET,SET/10 RED POLKADOT PARTY CANDLES,SET/10 BLUE POLKADOT PARTY CANDLES,REGENCY TEAPOT ROSES,JAM JAR WITH PINK LID,JAM JAR WITH GREEN LID,SET 6 SCHOOL MILK BOTTLES IN CRATE,PACK OF 20 NAPKINS PANTRY DESIGN,SET OF 12 MINI LOAF BAKING CASES,SET OF 6 TEA TIME BAKING CASES,EMBROIDERED RIBBON REEL DAISY,EMBROIDERED RIBBON REEL SOPHIE,EMBROIDERED RIBBON REEL ROSIE,SET 12 COLOURING PENCILS DOILY,JAM MAKING SET PRINTED,SET OF 6 SNACK LOAF BAKING CASES,SET OF 3 REGENCY CAKE TINS,REGENCY SUGAR BOWL GREEN,REGENCY MILK JUG PINK,JUMBO BAG 50'S CHRISTMAS,JUMBO BAG RED RETROSPOT,LUNCH BAG VINTAGE DOILY,RETRO PLASTIC ELEPHANT TRAY,SPACEBOY ROCKET LOLLY MAKERS,JUNGLE POPSICLES ICE LOLLY MOULDS,SET OF 5 LUCKY CAT MAGNETS,ELEPHANT BIRTHDAY CARD,VINTAGE KID DOLLY CARD,RED RETROS

In [27]:
def change_values(z):
  for i in prods:
    if z[i] > 0:
      z[i] = i

  return z 

In [28]:
df_filtered = df_filtered.apply(change_values, axis=1)
df_filtered.head()

,SET OF 6 SPICE TINS PANTRY DESIGN,PANTRY WASHING UP BRUSH,SET OF 36 TEATIME PAPER DOILIES,SET OF 3 CAKE TINS PANTRY DESIGN,SET OF 4 PANTRY JELLY MOULDS,TEA TIME DES TEA COSY,SET 2 PANTRY DESIGN TEA TOWELS,SET 2 TEA TOWELS I LOVE LONDON,PARTY CONES CANDY ASSORTED,PINK HAPPY BIRTHDAY BUNTING,3 TRADITIONAl BISCUIT CUTTERS SET,PAPER BUNTING COLOURED LACE,COTTON APRON PANTRY DESIGN,PARTY CONES CANDY TREE DECORATION,VINTAGE RED TEATIME MUG,FELTCRAFT 6 FLOWER FRIENDS,FELTCRAFT BUTTERFLY HEARTS,COFFEE MUG CAT + BIRD DESIGN,LONDON BUS COFFEE MUG,REGENCY CAKESTAND 3 TIER,TEA TIME TEA SET IN GIFT BOX,TEA TIME TEAPOT IN GIFT BOX,PLASTERS IN TIN SPACEBOY,RED HARMONICA IN BOX,SET OF 6 SOLDIER SKITTLES,BAKING SET 9 PIECE RETROSPOT,DOORMAT SPOTTY HOME SWEET HOME,RED STRIPE CERAMIC DRAWER KNOB,RED SPOT CERAMIC DRAWER KNOB,BLUE STRIPE CERAMIC DRAWER KNOB,PAPER BUNTING RETROSPOT,PINK BABY BUNTING,BLUE HAPPY BIRTHDAY BUNTING,CHILDRENS DOLLY GIRL MUG,CHILDREN'S SPACEBOY MUG,ROUND SNACK BOXES SET OF4 WOODLAND,SWEETHEART CERAMIC TRINKET BOX,STRAWBERRY CERAMIC TRINKET BOX,36 DOILIES DOLLY GIRL,SET OF 36 PAISLEY FLOWER DOILIES,SET OF 4 ENGLISH ROSE PLACEMATS,CERAMIC CAKE DESIGN SPOTTED MUG,STRAWBERRY FAIRY CAKE TEAPOT,60 CAKE CASES DOLLY GIRL DESIGN,PACK OF 60 MUSHROOM CAKE CASES,PACK OF 60 SPACEBOY CAKE CASES,PACK OF 72 RETROSPOT CAKE CASES,CUPCAKE LACE PAPER SET 6,60 TEATIME FAIRY CAKE CASES,FRYING PAN RED RETROSPOT,BREAD BIN DINER STYLE PINK,MILK PAN BLUE POLKADOT,GLASS JAR ENGLISH CONFECTIONERY,TEA TIME OVEN GLOVE,MAGNETS PACK OF 4 CHILDHOOD MEMORY,FRIDGE MAGNETS US DINER ASSORTED,SET OF 5 MINI GROCERY MAGNETS,SET OF 5 PANCAKE DAY MAGNETS,TEA PARTY BIRTHDAY CARD,SPACEBOY BIRTHDAY CARD,CARD BILLBOARD FONT,CARD DOLLY GIRL,WRAP ENGLISH ROSE,RED RETROSPOT WRAP,BLUE POLKADOT WRAP,TEA PARTY WRAPPING PAPER,BIRTHDAY BANQUET GIFT WRAP,PINK PAISLEY ROSE GIFT WRAP,WRAP DOLLY GIRL,WRAP COWBOYS,SET/10 PINK POLKADOT PARTY CANDLES,SET/10 IVORY POLKADOT PARTY CANDLES,TRADITIONAL WOODEN SKIPPING ROPE,MEDIUM CERAMIC TOP STORAGE JAR,LARGE CERAMIC TOP STORAGE JAR,SMALL CERAMIC TOP STORAGE JAR,DOORMAT FAIRY CAKE,SET 36 COLOUR PENCILS DOLLY GIRL,SET 12 COLOUR PENCILS SPACEBOY,EMBROIDERED RIBBON REEL SUSIE,EMBROIDERED RIBBON REEL RACHEL,EMBROIDERED RIBBON REEL SALLY,RETROSPOT TEA SET CERAMIC 11 PC,ICE CREAM PEN LIP GLOSS,DOUGHNUT LIP GLOSS,PARTY INVITES WOODLAND,PARTY INVITES DINOSAURS,MINI PAINT SET VINTAGE,RIBBON REEL SNOWY VILLAGE,SET OF 6 RIBBONS VINTAGE CHRISTMAS,RIBBON REEL CHRISTMAS PRESENT,3 RAFFIA RIBBONS 50'S CHRISTMAS,RIBBON REEL MAKING SNOWMEN,RIBBON REEL CHRISTMAS SOCK BAUBLE,WRAP CHRISTMAS VILLAGE,CARD CHRISTMAS VILLAGE,GLASS APOTHECARY BOTTLE ELIXIR,VINTAGE DONKEY TAIL GAME,DOLLY GIRL CHILDRENS BOWL,DOLLY GIRL CHILDRENS CUP,SPACEBOY CHILDRENS BOWL,SPACEBOY CHILDRENS CUP,SET OF 72 RETROSPOT PAPER DOILIES,6 GIFT TAGS VINTAGE CHRISTMAS,ROLL WRAP VINTAGE CHRISTMAS,ROLL WRAP 50'S CHRISTMAS,6 GIFT TAGS 50'S CHRISTMAS,VINTAGE CHRISTMAS STOCKING,SET OF 60 I LOVE LONDON CAKE CASES,SET OF 12 FAIRY CAKE BAKING CASES,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY SPACEBOY,CHILDRENS CUTLERY DOLLY GIRL,RED RETROSPOT MINI CASES,SILK PURSE BABUSHKA PINK,SAVE THE PLANET COTTON TOTE BAG,FAIRY CAKE BIRTHDAY CANDLE SET,SET/10 RED POLKADOT PARTY CANDLES,SET/10 BLUE POLKADOT PARTY CANDLES,REGENCY TEAPOT ROSES,JAM JAR WITH PINK LID,JAM JAR WITH GREEN LID,SET 6 SCHOOL MILK BOTTLES IN CRATE,PACK OF 20 NAPKINS PANTRY DESIGN,SET OF 12 MINI LOAF BAKING CASES,SET OF 6 TEA TIME BAKING CASES,EMBROIDERED RIBBON REEL DAISY,EMBROIDERED RIBBON REEL SOPHIE,EMBROIDERED RIBBON REEL ROSIE,SET 12 COLOURING PENCILS DOILY,JAM MAKING SET PRINTED,SET OF 6 SNACK LOAF BAKING CASES,SET OF 3 REGENCY CAKE TINS,REGENCY SUGAR BOWL GREEN,REGENCY MILK JUG PINK,JUMBO BAG 50'S CHRISTMAS,JUMBO BAG RED RETROSPOT,LUNCH BAG VINTAGE DOILY,RETRO PLASTIC ELEPHANT TRAY,SPACEBOY ROCKET LOLLY MAKERS,JUNGLE POPSICLES ICE LOLLY MOULDS,SET OF 5 LUCKY CAT MAGNETS,ELEPHANT BIRTHDAY CARD,VINTAGE KID DOLLY CARD,RED RETROS

In [29]:
basket_set = df_filtered.values
basket_set

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        'TEA PARTY BIRTHDAY CARD', 0, 0, 'CARD DOLLY GIRL', 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, "JUMBO BAG 50'S CHRISTMAS", 'JUMBO BAG RED RETROSPOT',
        'LUNCH BAG VINTAGE DOILY', 'RETRO PLASTIC ELEPHANT TRAY',
        'SPACEBOY ROCKET LOLLY MAKERS',
        'JUNGLE POPSICLES ICE LOLLY MOULDS',
        'SET OF 5 LUCKY CAT MAGNETS', 'ELEPHANT BIRTHDAY CARD',
        'VINTAGE KID DOLLY CARD', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 'SET OF 36 TEATIME PAPER DOILIES',
        'SET OF 3 CAKE TINS PANTRY DESIGN', 0, 0,
        'SET 2 PANTRY 

In [30]:
basket = [i[i!=0].tolist() for i in basket_set if i[i!=0].tolist()]
basket[:5]

[['TEA PARTY BIRTHDAY CARD',
  'CARD DOLLY GIRL',
  "JUMBO BAG 50'S CHRISTMAS",
  'JUMBO BAG RED RETROSPOT',
  'LUNCH BAG VINTAGE DOILY',
  'RETRO PLASTIC ELEPHANT TRAY',
  'SPACEBOY ROCKET LOLLY MAKERS',
  'JUNGLE POPSICLES ICE LOLLY MOULDS',
  'SET OF 5 LUCKY CAT MAGNETS',
  'ELEPHANT BIRTHDAY CARD',
  'VINTAGE KID DOLLY CARD'],
 ['SET OF 36 TEATIME PAPER DOILIES',
  'SET OF 3 CAKE TINS PANTRY DESIGN',
  'SET 2 PANTRY DESIGN TEA TOWELS',
  'PINK HAPPY BIRTHDAY BUNTING',
  'PLASTERS IN TIN SPACEBOY',
  'RED  HARMONICA IN BOX',
  'SET OF 6 SOLDIER SKITTLES',
  'BAKING SET 9 PIECE RETROSPOT',
  'DOORMAT SPOTTY HOME SWEET HOME',
  'RED STRIPE CERAMIC DRAWER KNOB',
  'RED SPOT CERAMIC DRAWER KNOB',
  'BLUE STRIPE CERAMIC DRAWER KNOB',
  'PAPER BUNTING RETROSPOT',
  'PINK BABY BUNTING',
  'BLUE HAPPY BIRTHDAY BUNTING',
  'CHILDRENS DOLLY GIRL MUG',
  "CHILDREN'S SPACEBOY MUG",
  'ROUND SNACK BOXES SET OF4 WOODLAND',
  'SWEETHEART CERAMIC TRINKET BOX',
  'STRAWBERRY CERAMIC TRINKET BOX',
  

## Apriori using apyori

In [53]:
association = apriori(basket, min_support = 0.0003, min_confidence = 0.05, max_length = 3, min_lift = 3)

associ_res = list(association)

associ_res

[RelationRecord(items=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT', 'BLUE POLKADOT BOWL'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT'}), items_add=frozenset({'BLUE POLKADOT BOWL'}), confidence=1.0, lift=5.0), OrderedStatistic(items_base=frozenset({'BLUE POLKADOT BOWL'}), items_add=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT'}), confidence=1.0, lift=5.0)]),
 RelationRecord(items=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT', 'BLUE POLKADOT CUP'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT'}), items_add=frozenset({'BLUE POLKADOT CUP'}), confidence=1.0, lift=5.0), OrderedStatistic(items_base=frozenset({'BLUE POLKADOT CUP'}), items_add=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT'}), confidence=1.0, lift=5.0)]),
 RelationRecord(items=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT', 'CHILDRENS CUTLERY POLKADOT BLUE'}), support=0.2, ordered_sta

In [56]:
type(associ_res[1])

apyori.RelationRecord

In [61]:
print(associ_res[2])

RelationRecord(items=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT', 'CHILDRENS CUTLERY POLKADOT BLUE'}), support=0.2, ordered_statistics=[OrderedStatistic(items_base=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT'}), items_add=frozenset({'CHILDRENS CUTLERY POLKADOT BLUE'}), confidence=1.0, lift=5.0), OrderedStatistic(items_base=frozenset({'CHILDRENS CUTLERY POLKADOT BLUE'}), items_add=frozenset({'12 PENCILS SMALL TUBE RED RETROSPOT'}), confidence=1.0, lift=5.0)])


## FPGrowth using pyfpgrowth

In [1]:
!pip install pyfpgrowth

import pyfpgrowth

     |████████████████████████████████| 1.6 MB 5.3 MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5504 sha256=27ef0700a9cb7814d89f0c4e7858d8901c1de9356e63adb3ebca793256f69187
  Stored in directory: /root/.cache/pip/wheels/73/97/4b/f12ac994f6bbb99597396255435824c73ad3916be1e678be55
Successfully built pyfpgrowth


In [31]:
patterns = pyfpgrowth.find_frequent_patterns(basket, 2)

In [32]:
patterns

{('CARD DOLLY GIRL',): 2,
 ('CARD DOLLY GIRL', 'TEA PARTY BIRTHDAY CARD'): 2,
 ('EMBROIDERED RIBBON REEL SUSIE',): 2,
 ('EMBROIDERED RIBBON REEL SUSIE', 'SET OF 12 FAIRY CAKE BAKING CASES'): 2,
 ('EMBROIDERED RIBBON REEL SUSIE',
  'SET OF 12 FAIRY CAKE BAKING CASES',
  'SET OF 6 RIBBONS VINTAGE CHRISTMAS'): 2,
 ('EMBROIDERED RIBBON REEL SUSIE',
  'SET OF 12 FAIRY CAKE BAKING CASES',
  'SET OF 6 RIBBONS VINTAGE CHRISTMAS',
  'SET/10 BLUE POLKADOT PARTY CANDLES'): 2,
 ('EMBROIDERED RIBBON REEL SUSIE',
  'SET OF 12 FAIRY CAKE BAKING CASES',
  'SET OF 6 RIBBONS VINTAGE CHRISTMAS',
  'SET/10 BLUE POLKADOT PARTY CANDLES',
  'SET/10 IVORY POLKADOT PARTY CANDLES'): 2,
 ('EMBROIDERED RIBBON REEL SUSIE',
  'SET OF 12 FAIRY CAKE BAKING CASES',
  'SET OF 6 RIBBONS VINTAGE CHRISTMAS',
  'SET/10 BLUE POLKADOT PARTY CANDLES',
  'SET/10 IVORY POLKADOT PARTY CANDLES',
  'SET/10 PINK POLKADOT PARTY CANDLES'): 2,
 ('EMBROIDERED RIBBON REEL SUSIE',
  'SET OF 12 FAIRY CAKE BAKING CASES',
  'SET OF 6 RIBBON

In [33]:
rules = pyfpgrowth.generate_association_rules(patterns, 0.7)

In [39]:
print(rules)

{('CARD DOLLY GIRL',): (('TEA PARTY BIRTHDAY CARD',), 1.0), ('TEA PARTY BIRTHDAY CARD',): (('CARD DOLLY GIRL',), 1.0), ('SET OF 3 CAKE TINS PANTRY DESIGN',): (('PINK HAPPY BIRTHDAY BUNTING', 'SET 2 PANTRY DESIGN TEA TOWELS', 'SET OF 36 TEATIME PAPER DOILIES'), 1.0), ('SET OF 36 TEATIME PAPER DOILIES',): (('PINK HAPPY BIRTHDAY BUNTING', 'SET 2 PANTRY DESIGN TEA TOWELS', 'SET OF 3 CAKE TINS PANTRY DESIGN'), 1.0), ('SET 2 PANTRY DESIGN TEA TOWELS',): (('PINK HAPPY BIRTHDAY BUNTING', 'SET OF 3 CAKE TINS PANTRY DESIGN', 'SET OF 36 TEATIME PAPER DOILIES'), 1.0), ('SET 2 PANTRY DESIGN TEA TOWELS', 'SET OF 3 CAKE TINS PANTRY DESIGN'): (('PINK HAPPY BIRTHDAY BUNTING', 'SET OF 36 TEATIME PAPER DOILIES'), 1.0), ('SET 2 PANTRY DESIGN TEA TOWELS', 'SET OF 36 TEATIME PAPER DOILIES'): (('PINK HAPPY BIRTHDAY BUNTING', 'SET OF 3 CAKE TINS PANTRY DESIGN'), 1.0), ('SET OF 3 CAKE TINS PANTRY DESIGN', 'SET OF 36 TEATIME PAPER DOILIES'): (('PINK HAPPY BIRTHDAY BUNTING', 'SET 2 PANTRY DESIGN TEA TOWELS'), 1.

## Apriori and FPGrowth using mlxtend

Dataset - [Groceries](https://www.kaggle.com/heeraldedhia/groceries-dataset)

In [5]:
!pip install mlxtend --upgrade

In [6]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

In [7]:
df_grocery = pd.read_csv('/content/drive/MyDrive/CMPE255 - Data Mining/Assignments/Datasets/Groceries_dataset.csv')

In [8]:
df_grocery.shape

(38765, 3)

In [9]:
df_grocery["itemDescription"].unique()

array(['tropical fruit', 'whole milk', 'pip fruit', 'other vegetables',
       'rolls/buns', 'pot plants', 'citrus fruit', 'beef', 'frankfurter',
       'chicken', 'butter', 'fruit/vegetable juice',
       'packaged fruit/vegetables', 'chocolate', 'specialty bar',
       'butter milk', 'bottled water', 'yogurt', 'sausage', 'brown bread',
       'hamburger meat', 'root vegetables', 'pork', 'pastry',
       'canned beer', 'berries', 'coffee', 'misc. beverages', 'ham',
       'turkey', 'curd cheese', 'red/blush wine',
       'frozen potato products', 'flour', 'sugar', 'frozen meals',
       'herbs', 'soda', 'detergent', 'grapes', 'processed cheese', 'fish',
       'sparkling wine', 'newspapers', 'curd', 'pasta', 'popcorn',
       'finished products', 'beverages', 'bottled beer', 'dessert',
       'dog food', 'specialty chocolate', 'condensed milk', 'cleaner',
       'white wine', 'meat', 'ice cream', 'hard cheese', 'cream cheese ',
       'liquor', 'pickled vegetables', 'liquor (appetizer

In [10]:
df_grocery['itemDescription'] = df_grocery.groupby(['Member_number'])['itemDescription'].transform(lambda x: ','.join(x))
df_grocery

,Member_number,Date,itemDescription
0,1808,21-07-2015,"tropical fruit,long life bakery product,meat,s..."
1,2552,05-01-2015,"whole milk,butter,female sanitary products,pot..."
2,2300,19-09-2015,"pip fruit,sausage,frankfurter,pork,other veget..."
3,1187,12-12-2015,"other vegetables,soda,citrus fruit,onions,soda..."
4,3037,01-02-2015,"whole milk,cling film/bags,other vegetables,ha..."
...,...,...,...
38760,4471,08-10-2014,"coffee,whole milk,whole milk,butter,yogurt,cre..."
38761,2022,23-02-2014,"herbs,frankfurter,sausage,rolls/buns,pork,frui..."
38762,1097,16-04-2014,"tropical fruit,frankfurter,whole milk,whipped/..."
38763,1510,03-12-2014,"sausage,ham,other vegetables,other vegetables,..."


In [11]:
transc_list=[]
for index, row in df_grocery.iterrows():   
    val = row["itemDescription"].split(',')
    transc_list.append(val)

In [12]:
transc_list

[['tropical fruit',
  'long life bakery product',
  'meat',
  'sugar',
  'rolls/buns',
  'semi-finished bread',
  'whole milk',
  'citrus fruit',
  'candy',
  'napkins'],
 ['whole milk',
  'butter',
  'female sanitary products',
  'pot plants',
  'other vegetables',
  'tropical fruit',
  'root vegetables',
  'whole milk',
  'shopping bags',
  'chocolate',
  'chocolate',
  'coffee',
  'hygiene articles'],
 ['pip fruit',
  'sausage',
  'frankfurter',
  'pork',
  'other vegetables',
  'white wine',
  'long life bakery product',
  'fruit/vegetable juice',
  'flour',
  'hygiene articles'],
 ['other vegetables',
  'soda',
  'citrus fruit',
  'onions',
  'soda',
  'domestic eggs',
  'shopping bags'],
 ['whole milk', 'cling film/bags', 'other vegetables', 'ham', 'white bread'],
 ['rolls/buns',
  'rolls/buns',
  'canned beer',
  'bottled beer',
  'ham',
  'coffee',
  'whole milk',
  'liquor',
  'yogurt',
  'white wine',
  'meat',
  'citrus fruit',
  'hamburger meat',
  'pickled vegetables',
  '

In [16]:
transcencd = TransactionEncoder()
te_data = transcencd.fit(transc_list).transform(transc_list)
df_trans = pd.DataFrame(te_data, columns=transcencd.columns_)
df_trans.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False
2,False,Fa

In [18]:
freq_items = apriori(df_trans, min_support=0.02,use_colnames=True)
freq_items.head()

,support,itemsets
0,0.100245,(UHT-milk)
1,0.040681,(baking powder)
2,0.149207,(beef)
3,0.100916,(berries)
4,0.078963,(beverages)


In [19]:
rules = association_rules(freq_items, metric = "confidence", min_threshold = 0.1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6441,"(rolls/buns, yogurt, brown bread)",(whole milk),0.030362,0.553308,0.024120,0.794393,1.435714,0.007320,2.172546
6039,"(brown bread, bottled water, other vegetables)",(whole milk),0.028067,0.553308,0.022056,0.785846,1.420267,0.006527,2.085839
7933,"(yogurt, root vegetables, shopping bags)",(whole milk),0.027912,0.553308,0.021798,0.780961,1.411439,0.006354,2.039326
6524,"(tropical fruit, canned beer, other vegetables)",(whole milk),0.028711,0.553308,0.022211,0.773585,1.398108,0.006324,1.972888
3182,"(pip fruit, chocolate)",(whole milk),0.031111,0.553308,0.023604,0.758706,1.371218,0.006390,1.851237
6028,"(rolls/buns, bottled beer, yogurt)",(whole milk),0.033277,0.553308,0.025203,0.757364,1.368792,0.006790,1.840997
4322,"(yogurt, ham)",(whole milk),0.031136,0.553308,0.023526,0.755592,1.365590,0.006298,1.827650
2500,"(brown bread, curd)",(whole milk),0.032478,0.553308,0.024532,0.755361,1.365172,0.006562,1.825924
7713,"(yogurt, pork, soda)",(whole milk),0.029356,0.553308,0.022133,0.753954,1.362629,0.005890,1.815482
6472,"(yogurt, butter, other vegetables)",(whole milk),0.037147,0.553308,0.027809,0.748611,1.352973,0.007255,1.776895


In [20]:
fp_itemsset = fpgrowth(df_trans, min_support=0.02,use_colnames=True)
fp_itemsset.head(10)

,support,itemsets
0,0.553308,(whole milk)
1,0.425435,(rolls/buns)
2,0.284174,(tropical fruit)
3,0.225229,(citrus fruit)
4,0.101174,(napkins)
5,0.088998,(sugar)
6,0.083219,(long life bakery product)
7,0.080769,(meat)
8,0.069392,(candy)
9,0.046563,(semi-finished bread)


In [21]:
associ_rules = association_rules(fp_itemsset, metric = "confidence", min_threshold = 0.1)
associ_rules.sort_values('confidence', ascending = False, inplace = True)
associ_rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5929,"(rolls/buns, yogurt, brown bread)",(whole milk),0.030362,0.553308,0.024120,0.794393,1.435714,0.007320,2.172546
5721,"(brown bread, bottled water, other vegetables)",(whole milk),0.028067,0.553308,0.022056,0.785846,1.420267,0.006527,2.085839
902,"(yogurt, root vegetables, shopping bags)",(whole milk),0.027912,0.553308,0.021798,0.780961,1.411439,0.006354,2.039326
4489,"(tropical fruit, canned beer, other vegetables)",(whole milk),0.028711,0.553308,0.022211,0.773585,1.398108,0.006324,1.972888
1734,"(pip fruit, chocolate)",(whole milk),0.031111,0.553308,0.023604,0.758706,1.371218,0.006390,1.851237
4613,"(rolls/buns, bottled beer, yogurt)",(whole milk),0.033277,0.553308,0.025203,0.757364,1.368792,0.006790,1.840997
4022,"(yogurt, ham)",(whole milk),0.031136,0.553308,0.023526,0.755592,1.365590,0.006298,1.827650
7853,"(brown bread, curd)",(whole milk),0.032478,0.553308,0.024532,0.755361,1.365172,0.006562,1.825924
3175,"(yogurt, pork, soda)",(whole milk),0.029356,0.553308,0.022133,0.753954,1.362629,0.005890,1.815482
1423,"(yogurt, butter, other vegetables)",(whole milk),0.037147,0.553308,0.027809,0.748611,1.352973,0.007255,1.776895


# References

1. https://fp-growth.readthedocs.io/en/latest/readme.html
2. http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#frequent-itemsets-via-apriori-algorithm

3. https://pypi.org/project/apyori/